In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

In [ ]:
df_columns_order = [
    'DummyOneString',
    'DummyFirstString',
    'NonThreadSafe',
    'SynchronizedMethod',
    'SynchronizedBlock',
    'Semaphore',
    'ReentrantLock',
    'AtomicIntegerCASet',
    'AtomicIntegerCAExchange',
    'AtomicIntegerLambda'
]

result_pattern = re.compile(r"""
    BalancersBenchmark._(?P<threads>\d*)_threads?\s*
    (?P<balancer>\w*)Balancer\s*
    (thrpt|avgt)\s*
    (?P<count>\d*)\s*
    (?P<score>\d*[.,]\d*)\s*
    ±\s*
    (?P<error>\d*[.,]\d*)\s*
""", re.VERBOSE)

def read_data(file):
    results = [];
    with open(file, 'r', encoding='utf-8') as input:
        in_summary = False
        for line in input:
            in_summary = in_summary or 'Run complete' in line
            if in_summary:
                match = result_pattern.match(line)
                if match:
                    row = match.groupdict()
                    row['score'] = row['score'].replace(',', '.')
                    row['error'] = row['error'].replace(',', '.')
                    results.append(row)

    data_long = pd.DataFrame.from_records(results)
    balancers = data_long['balancer'].unique();
    print(f'{file}:\n {balancers}\n')
    
    columns = df_columns_order
    [columns.append(x) for x in list(balancers) if x not in columns]
    
    data = pd.pivot_table(data_long, index='threads', columns='balancer', values='score')
    data.index = data.index.astype(int)
    data = data.sort_index().reindex(columns=columns)
    return data

dataset = 'windows-i5-8400-jdk-17.0.8'
thrpt_donoop = read_data(dataset + '/thrpt_donoop.txt')
thrpt_toupper = read_data(dataset + '/thrpt_toupper.txt')

avgt_donoop = read_data(dataset + '/avgt_donoop.txt')
avgt_toupper = read_data(dataset + '/avgt_toupper.txt')

thrpt_donoop_biased_locking = read_data(dataset + '/thrpt_donoop_biased-locking.txt')

In [ ]:
grid_alpha=0.4
data_colors = {
    'DummyOneString': '#14bae1',
    'DummyFirstString': '#2379cc',
    'NonThreadSafe': '#f73931',
    'SynchronizedMethod': '#15b71e',
    'SynchronizedBlock': '#78b98f',
    'Semaphore': '#a7d64e',
    'ReentrantLock': '#a8b8e6',
    'AtomicIntegerCASet': '#ff7f00',
    'AtomicIntegerCAExchange': '#fdbf6f',
    'AtomicIntegerLambda': '#fb9a99',
}

def preetify_line_plot(df, ax):
    ax.set_xticks(df.index)
    ax.grid('on', alpha=0.4)
    
def preetify_bar_plot(df, ax):
    ax.grid('on', axis='y', alpha=0.4)

def select(df, balancers=None, threads=None):
    if balancers:
        df = df.filter(regex=balancers)
    if threads:
        df = df.loc[set(df.index).intersection(range(threads[0], threads[1]+1))]
    return df

def plot_sel(df, balancers=None, threads=None, title=None, ylabel='total throughput [ops/μs]'):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    
    df = select(df, balancers, threads)    
    df.plot.line(ax=axes[0], title=title, ylabel=ylabel, color=data_colors)
    df.plot.bar(ax=axes[1], title=title, ylabel=ylabel, color=data_colors)
    preetify_line_plot(df, axes[0])
    preetify_bar_plot(df, axes[1])
    
def plot_cmp(df1, df2, balancers=None, threads=None, title1=None, title2=None, ylabel1=None, ylabel2=None,
             ylabel='total throughput [ops/μs]', type='line'):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    if not ylabel1:
        ylabel1 = ylabel
    if not ylabel2:
        ylabel2 = ylabel
    
    df1 = select(df1, balancers, threads)
    df2 = select(df2, balancers, threads)
    if type == 'line':
        df1.plot.line(ax=axes[0], title=title1, ylabel=ylabel1, color=data_colors)
        df2.plot.line(ax=axes[1], title=title2, ylabel=ylabel2, color=data_colors)
        preetify_line_plot(df1, axes[0])
        preetify_line_plot(df2, axes[1])
    else:
        df1.plot.bar(ax=axes[0], title=title1, ylabel=ylabel1, color=data_colors)
        df2.plot.bar(ax=axes[1], title=title2, ylabel=ylabel2, color=data_colors)
        preetify_bar_plot(df1, axes[0])
        preetify_bar_plot(df2, axes[1])
    
plot_sel(thrpt_donoop, balancers='(.*)', threads=(1,10), title='Plot Test')
plot_cmp(thrpt_donoop, thrpt_toupper, balancers='Dummy', title1='Plot Test X', title2='Plot Test Y', type='bar')

In [ ]:
plot_sel(thrpt_donoop, '(Dummy|Non.*Safe|Synch.*Method)', threads=(1,12), title='Basic Implementations - Throughput')

In [ ]:
#plot_sel(avgt_donoop, '(Dummy|Non.*Safe|Synch.*Method)', threads=(1,12), title='Basic Implementations - Average Time', ylabel='average time [ns/op]')
plot_sel(avgt_donoop, '(Dummy)', threads=(1,12), title='Dummy Implementations - Average Time', ylabel='average time [ns/op]')
plot_cmp(thrpt_donoop, avgt_donoop, '(Non.*Safe|Synch.*Method)', threads=(1,12),
         title1='Non-Dummy Implementations - Throughput', ylabel1='total throughput [ops/μs]',
         title2='Non-Dummy Implementations - Average Time', ylabel2='average time [ns/op]')

In [ ]:
plot_sel(thrpt_donoop, '(Dummy|Non.*Safe|Synch.*Method)', threads=(1,3), title='One vs Many Threads Slowdown')

In [ ]:
plot_cmp(thrpt_donoop, thrpt_donoop_biased_locking, '(Non.*Safe|Synch.*Method)', threads=(1,6),
         title1='Without Biased Locking', title2='With Biased Locking', type='bar')

In [ ]:
plot_sel(thrpt_donoop, '(Non.*Safe|Synch|Semaph|Lock)', threads=(1,6), title='Thread Safe Implementations - Lock Based')

In [ ]:
plot_sel(thrpt_donoop, '(Non.*Safe|Atomic)', threads=(1,6), title='Thread Safe Implementations - CAS Based')

In [ ]:
plot_sel(thrpt_donoop, '(Synch|Semaph|Lock|Atomic)', threads=(1,6), title='Lock-Based vs. CAS-Based')